In [1]:
import pandas as pd
import numpy as np

In [2]:
# Mesic pro ktery se operace provadi
month = '09_2019'
# Format souboru, napr ENGS_08_2019.xlsx
xls = pd.ExcelFile("ENGS_"+month+".xlsx")
# Lide, pro ktere hledame smlouvy
people_names = pd.read_excel(xls, 'список МТО')
# Tabulka smluv prirazenych k lidem
contract_people = pd.read_excel(xls, 'договора')
# vsechny tri listy v excelu, zacinajici na tretim radku
data_tuple = (pd.read_excel(xls, '60.01', header=2), pd.read_excel(xls, '60.21', header=2), pd.read_excel(xls, '60.31', header=2))

In [3]:
# Vsechny mozne meny
currencies = ['USD', 'EUR', 'CNY', 'JPY', 'GBP', 'руб']

In [4]:
# Vsechny mozne jmena firem, prectene z druheho sloupce
company_names = list(set(contract_people.iloc[:,1].values))
# Odstraneni prazdnych bunek
company_names = [name for name in company_names if str(name) != 'nan']

In [5]:
def add_company(data):
    last_company = ""
    for row in range(len(data)):
        if data.loc[row, "Счет"] in company_names:
            last_company = data.loc[row, "Счет"]
        data.loc[row, "Firma"] = last_company
    return data

In [6]:
def add_contract_nr(data):  
    last_contract_nr = ""
    for row in range(len(data)):
        if (type(data.loc[row, "Счет"]) is not str and np.isnan(data.loc[row, "Счет"])) or data.loc[row, "Счет"] in currencies:
            data.loc[row, "Smlouva"] = last_contract_nr
        else:
            last_contract_nr = data.loc[row, "Счет"]
    return data

In [7]:
def filter_by_turnover(data):
    final_rows = []
    for row in range(len(data)):
        if data.iloc[row,1] == 'Оборот':
            if not np.isnan(data.iloc[row,3]):
                final_rows.append(row)
    return data.iloc[final_rows, :]

In [8]:
def add_currency(data):
    last_currency = ""
    for row in range(len(data)):
        if data.loc[row, "Счет"] in currencies:
            last_currency = data.loc[row, "Счет"]
        data.loc[row, "Mena"] = last_currency    
    return data

In [9]:
def remove_space(x):
    while type(x) is str and x.endswith(" "):
        x = x[:-1]
    return x

In [10]:
contracts = contract_people[contract_people['Подготовил'].isin(people_names.values.flatten())].copy()
contracts.rename(columns={ 'Наименование': "Smlouva", 'Подготовил': "Osoba"}, inplace = True)
contracts = contracts[['Smlouva','Osoba']]
contracts = contracts.applymap(remove_space)

## Main for loop

In [11]:
file_names = ['01', '21', '31']
for i in range(3):
    data = data_tuple[i].copy()
    data.rename(columns={ data.columns[3]: "Castka", data.columns[4]: "Firma", data.columns[5]: "Smlouva", data.columns[6]: "Mena"}, inplace = True)
    data = add_company(data)
    data = add_contract_nr(data)
    data = add_currency(data)
    data = filter_by_turnover(data)
    data.loc[:,"Firma":"Mena"] = data.loc[:,"Firma":"Mena"].applymap(remove_space)
    data = data.merge(contracts, on = "Smlouva")
    data = data[["Firma", "Smlouva", "Castka", "Mena", "Osoba"]]
    if file_names[i] == '01':
        data['Mena'] = 'руб'
    last_company = ""
    company_sum = 0
    rows_to_drop = []
    for row in range(len(data)):
        if data.loc[row,"Firma"] != last_company:
            if row > 0 and company_sum/2.01 <= data.loc[row - 1,"Castka"] <= company_sum/1.99:
                rows_to_drop.append(row-1)
            company_sum = data.loc[row,"Castka"]
            last_company = data.loc[row,"Firma"]
        else:
            company_sum += data.loc[row,"Castka"]
        if row == len(data) -1:
            if company_sum/2.01 <= data.loc[row,"Castka"] <= company_sum/1.99:
                rows_to_drop.append(row)
    data_dropped = data.drop(rows_to_drop, axis=0)
    data_dropped.drop_duplicates(inplace=True)
    data_dropped.to_excel("ENGS_MTO_"+month+"_"+file_names[i]+".xlsx", engine='xlsxwriter')
    display(data_dropped)

,Firma,Smlouva,Castka,Mena,Osoba
0,Аврора ООО,Договор поставки № ТОА-3103 от 31.03.16г.,2888000.00,руб,Мен Виктория
1,Аврора ООО,Приложение №41 к Договору поставки № ТОА-3103 ...,454300.00,руб,Мен Виктория
3,Альфамонолит ООО,Спецификация № 3 кДоговору № 13486 от 27.02.20...,657900.00,руб,Мурадимов Акрам
5,Гранула ООО,Спецификация № 14 к Договору № 20160330/01ГР о...,4164437.55,руб,Мурадимов Акрам
6,Гранула ООО,Спецификация №18 к Договору №20160330/01ГР от ...,1076712.00,руб,Мурадимов Акрам
7,Гранула ООО,Спецификация №19 от 19.06.2019г. к Договору № ...,6845866.20,руб,Мурадимов Акрам
9,ИнБурТех ООО,Спецификация № 83 к Договору № ЭНГ00-06548 о...,506833.32,руб,Мухаммадиев Анвар
10,Кварт ООО,Договор №21 от 23.07.2019,39719.65,руб,Мухаммадиев Анвар
11,Кварт ООО,Спецификация 2 к Договору №21 от 23.07.2019,186705.58,руб,Мухаммадиев Анвар
13,Краснодарский завод Нефтемаш,Приложение №12 к Договору поставки №ТО0505.16 ...,152560.00,руб,Мен Виктория


,Firma,Smlouva,Castka,Mena,Osoba
0,"Jiangsu Petroleum Machinery Co.,Ltd",Приложение №15 к Контракту № ER-JPM-1 dd 16.1...,994760.00,CNY,Бу Владислав
2,"Jiangsu Petroleum Machinery Co.,Ltd",Приложение №13 к Контракту № ER-JPM-1 dd 16.10...,994760.00,CNY,Бу Владислав
3,NBS SOLUTIONS AS,Контракт №ЭНГ00-51659 от 23.08.2019г.,135999.24,EUR,Бакридинов Нурали


,Firma,Smlouva,Castka,Mena,Osoba
